# Test Model

In [1]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.getcwd()).parent))

from settings.global_settings import GlobalSettings

config = GlobalSettings.get_config(
    config_file = "../config.ini",
    secrets_file = "../secrets.ini"
)
from dataset.video_loader import VideoDataLoader
from dataset.video_dataset import VideoDataset
from model.training_loop import train
from torch.utils.data import random_split
from model.multimodal_har_model import MultiModalHARModel

2025-10-19 22:26:10,861 - INFO - Sentry DSN set to: https://f4f21cc936b3ba9f5dbc1464b7a40ea4@o4504168838070272.ingest.us.sentry.io/4506464560414720
2025-10-19 22:26:10,862 - INFO - Sentry initialized with environment: development


Loading config...
Loading secrets...


## Initializing Training

**Creating Dataloaders**

In [2]:
video_data_loader = VideoDataLoader(
    path=config.model_settings.video_data_dir
)

video_dataset = VideoDataset(
    video_data_loader=video_data_loader,
    normalization_type="across_frames",
)

len(video_dataset)
for _ in video_dataset:
    pass
len(video_dataset.labels_map)

2025-10-19 22:26:12,727 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-10-19 22:26:13,199 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-10-19 22:26:13,667 - INFO - [VideoDataLoader] Loding action videos for action: a03
2025-10-19 22:26:14,171 - INFO - [VideoDataLoader] Loding action videos for action: a04
2025-10-19 22:26:14,792 - INFO - [VideoDataLoader] Loding action videos for action: a05
2025-10-19 22:26:15,097 - INFO - [VideoDataLoader] Loding action videos for action: a06
2025-10-19 22:26:15,589 - INFO - [VideoDataLoader] Loding action videos for action: a08
2025-10-19 22:26:16,769 - INFO - [VideoDataLoader] Loding action videos for action: a09
2025-10-19 22:26:17,595 - INFO - [VideoDataLoader] Loding action videos for action: a11
2025-10-19 22:26:17,953 - INFO - [VideoDataLoader] Loding action videos for action: a12


10

**Splitting Train and Test Datasets**

In [3]:
num_total = len(video_dataset)
num_train = int(0.8 * num_total)
num_test = num_total - num_train
train_dataset, test_dataset = random_split(video_dataset, [num_train, num_test])

**Creating Model**

In [4]:

har_model = MultiModalHARModel(
    obj_in=train_dataset[0].graphs_objects[0].x.shape[1],
    joint_in=train_dataset[0].graphs_joints[0].x.shape[1],
    gat_hidden=128,
    gat_out=128,
    temporal_hidden=128,
    num_classes=len(video_dataset.labels_map), 
)

## Training Model

In [5]:
train_history = train(
    model=har_model,
    video_dataset=train_dataset,
    device='mps',
    epochs=25,
)

2025-10-19 22:26:23,390 - INFO - Starting training loop...
Epoch 1/25: 100%|██████████| 1180/1180 [06:02<00:00,  3.25it/s]
2025-10-19 22:32:26,279 - INFO - Epoch 1/25, Loss: 2.0920
Epoch 2/25: 100%|██████████| 1180/1180 [07:06<00:00,  2.77it/s]
2025-10-19 22:39:32,519 - INFO - Epoch 2/25, Loss: 1.6218
Epoch 3/25: 100%|██████████| 1180/1180 [05:53<00:00,  3.33it/s]
2025-10-19 22:45:26,389 - INFO - Epoch 3/25, Loss: 1.2853
Epoch 4/25: 100%|██████████| 1180/1180 [05:50<00:00,  3.36it/s]
2025-10-19 22:51:17,319 - INFO - Epoch 4/25, Loss: 1.0693
Epoch 5/25: 100%|██████████| 1180/1180 [08:19<00:00,  2.36it/s]
2025-10-19 22:59:36,453 - INFO - Epoch 5/25, Loss: 0.9750
Epoch 6/25: 100%|██████████| 1180/1180 [05:55<00:00,  3.32it/s]
2025-10-19 23:05:31,833 - INFO - Epoch 6/25, Loss: 0.9366
Epoch 7/25: 100%|██████████| 1180/1180 [06:55<00:00,  2.84it/s]
2025-10-19 23:12:27,409 - INFO - Epoch 7/25, Loss: 0.8637
Epoch 8/25: 100%|██████████| 1180/1180 [21:17<00:00,  1.08s/it]   
2025-10-19 23:33:45,

**Saving Model**

In [9]:
har_model.save(
    training_history=train_history
)

PermissionError: [Errno 13] Permission denied: '/Volumes/KODAK'

In [11]:
test_video_indices = test_dataset.indices
print("Test video IDs:", test_video_indices)

Test video IDs: [577, 711, 1456, 507, 730, 1338, 1391, 1231, 82, 936, 1109, 828, 1085, 320, 34, 928, 894, 1459, 1092, 681, 195, 874, 790, 947, 582, 1469, 1050, 1197, 788, 597, 651, 906, 429, 288, 870, 69, 913, 1218, 1467, 377, 1196, 306, 345, 1246, 2, 1323, 735, 25, 1033, 223, 1157, 1052, 756, 1139, 41, 1156, 548, 90, 245, 233, 30, 1006, 485, 129, 1321, 197, 274, 1272, 32, 295, 591, 601, 277, 558, 1160, 198, 360, 1145, 24, 823, 701, 87, 391, 381, 1305, 1453, 143, 879, 852, 643, 1229, 1161, 83, 1055, 176, 1174, 1443, 1306, 204, 476, 1214, 467, 141, 665, 555, 666, 1049, 934, 1136, 1032, 133, 1007, 411, 227, 1155, 148, 762, 647, 519, 230, 441, 238, 987, 28, 254, 528, 367, 858, 1090, 343, 1149, 100, 917, 324, 1037, 640, 985, 147, 7, 1281, 825, 406, 206, 1142, 1026, 539, 127, 1285, 1372, 219, 603, 220, 363, 963, 524, 316, 1242, 1030, 104, 573, 124, 1201, 1056, 779, 27, 527, 202, 943, 214, 225, 1217, 621, 915, 252, 759, 976, 792, 160, 265, 134, 996, 1432, 543, 1184, 1337, 859, 287, 950, 1328

In [12]:
import torch
from datetime import datetime

model_settings = config.model_settings
save_dir = "~/masters-models"
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(
    save_dir,
    f"har_model_{model_settings.model_version}_{model_settings.dataset_prefix}_100perc_{datetime.now()}.pht"
)
torch.save({
"model_state_dict": har_model.state_dict(),
"training_history": train_history,
"model_config": har_model._model_config,
"test_set_indices": test_video_indices,
}, save_path)


## Running tests

**Accuracy**

In [21]:
def evaluate(model, dataset, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i in range(len(dataset)):
            sample = dataset[i]
            label = sample.label.to(device)

            # Move all graph tensors to device
            graphs_objects = [g.to(device) for g in sample.graphs_objects]
            graphs_joints = [g.to(device) for g in sample.graphs_joints]

            # Forward pass
            output = model(graphs_objects, graphs_joints)

            # Compute prediction
            if output.dim() == 1:
                predicted = torch.argmax(output).unsqueeze(0)
            else:
                _, predicted = torch.max(output, dim=1)

            correct += (predicted == label).sum().item()
            total += 1

    accuracy = 100 * correct / total
    return accuracy


In [24]:
accuracy_evaluation = evaluate(har_model, test_dataset, "mps")
print(f"Test Accuracy: {accuracy_evaluation:.2f}%")

Test Accuracy: 68.14%
